# Image Recognition Walkthrough

Today we are going to work with Cats and Dogs data because Cats and Dogs are fun. :) 

Let's remember the steps of a ML workflow:
- Collecting data
- Importing libraries and splitting data to train and test
- Building the network
- Data augmentation
- Training
- Testing

Step 1 - Collecting Data:  
https://drive.google.com/file/d/1FTCTZaNkgmAyoQ2pZYR4LKxcXI4bLp00/view

Step 2: Loading the libraries

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

Step 3 - Building a CNN

Today, we will have a simple architecture. 
It consists of four layers:

1. Convolution
2. Pooling
3. Flattening
4. Full connection

In [3]:
# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Convolution2D(32, 3, 3, input_shape = (64, 64, 3), activation = 'relu'))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Step 3 - Flattening
classifier.add(Flatten())
# Step 4 - Full connection
classifier.add(Dense(128, activation = 'relu'))
classifier.add(Dense(1, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

1. Convolution: Extract features from the input image. 
    * Filters: The dimensionality of the output space (number of output filters in the convolution). 
    * Kernel_size: Height and width of the 2D convolution window.
    * Strides: Side of the strides of the convolution along with the height and width. 
      
      
      
2. Pooling: Define a spatial neighborhood (in our case a 2×2 window) and take the largest element from the rectified feature map within that window.


3. Flattening: Convert the matrix into a 1D array which can be the input of the final Neural Network.


4. Full Connection: Connect our convolutional network to a Neural Network that does final predictions.

Step 4 - Data Augmentation

Suppose we have a limited number of images for our network – what do we do? We don’t need to hunt for new images that can be added to our dataset. 

We can make alterations to our existing dataset – minor changes such as flips, translations, or rotations – and our neural network will think these are distinct images anyway.

Data augmentation is a way of reducing overfitting of models, where we increase the amount of training data using only the information from our training data.

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


Fitting the model

Play with the number of epochs and batches. The current model is slow but performs "reasonably well" if you do not have a GPU.

With an increasing number of epochs, the accuracy will increase, too.

In [5]:
history = classifier.fit_generator(training_set,
                         steps_per_epoch = 70,
                         epochs = 15,
                         validation_data = test_set)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
70/70 [==============================] - 18s 255ms/step - loss: 0.6928 - accuracy: 0.5402 - val_loss: 0.6715 - val_accuracy: 0.6120
Epoch 2/15
70/70 [==============================] - 14s 194ms/step - loss: 0.6757 - accuracy: 0.5835 - val_loss: 0.6573 - val_accuracy: 0.5875
Epoch 3/15
70/70 [==============================] - 13s 181ms/step - loss: 0.6415 - accuracy: 0.6246 - val_loss: 0.6253 - val_accuracy: 0.6540
Epoch 4/15
70/70 [==============================] - 12s 172ms/step - loss: 0.6123 - accuracy: 0.6687 - val_loss: 0.6104 - val_accuracy: 0.6665
Epoch 5/15
70/70 [==============================] - 12s 174ms/step - loss: 0.6086 - accuracy: 0.6732 - val_loss: 0.5904 - val_accuracy: 0.6900
Epoch 6/15
70/70 [==============================] - 12s 170ms/step - loss: 0.6026 - accuracy: 0.6558 - val_loss: 0.6072 - val_accuracy: 0.6595
Epoch 7/15
70/70 [==============================] - 12s 168ms/step

Step 5. Evaluating the model

In [6]:
from matplotlib import pyplot as plt
# plot history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

<Figure size 640x480 with 1 Axes>

Observations: 

Let's try something fun. This is my own personal cat and my own personal dog. Let's see if the network says anything about them.

![mypet](dataset/mypet1.jpg)

In [10]:
import numpy as np
from tensorflow.keras.preprocessing import image
# loading an image from the disk
test_image = image.load_img('dataset/mycat2.jpg', target_size = (64, 64))
# converting the image to a numpy array
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
# print(training_set.class_indices)
# our cut-off
if result[0][0] > 0.5:
    prediction = 'dog'
else:
    prediction = 'cat'
print(prediction)

cat
